# Fungut

# 00 Packages and Directory

In [2]:
import IPython

import pandas as pd
import matplotlib.pyplot as plt
import qiime2 as q2
import seaborn as sns
from qiime2 import Visualization


import os

import matplotlib.pyplot as plt
%matplotlib inline

# 01 Data import

In [3]:
PATH = "fungut_data/fungut_metadata.tsv"

In [4]:
surveys_df = pd.read_csv(PATH, sep="\t")

In [5]:
data_dir = 'fungut_data'

In [7]:
!qiime tools peek $data_dir/fungut_forward_reads.qza

UUID:        3638611d-1767-413b-9390-70ee3d78e4ff
Type:        SampleData[SequencesWithQuality]
Data format: SingleLanePerSampleSingleEndFastqDirFmt


In [8]:
!qiime demux summarize \
  --i-data $data_dir/fungut_forward_reads.qza \
  --o-visualization $data_dir/demux_summary.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: fungut_data/demux_summary.qzv


In [9]:
Visualization.load(f"{data_dir}/demux_summary.qzv")

<visualization: Visualization uuid: f14c242f-4d16-44a9-b651-4b64f42c1d40>

# 02 Trimming the primers

In [8]:
!qiime cutadapt trim-single \
  --i-demultiplexed-sequences $data_dir/fungut_forward_reads.qza \
  --p-front CTTGGTCATTTAGAGGAAGTAA \
  --o-trimmed-sequences $data_dir/fungut_forward_reads_trimmed.qza \
  --verbose

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: cutadapt -u 0 --error-rate 0.1 --times 1 --overlap 3 --minimum-length 1 -q 0,0 --quality-base 33 --cores 1 -o /tmp/qiime2/jovyan/processes/9805-1761647670.19@jovyan/tmp/q2-OutPath-it4lyt0k/ERR5327198_01_L001_R1_001.fastq.gz --front CTTGGTCATTTAGAGGAAGTAA /tmp/qiime2/jovyan/data/3638611d-1767-413b-9390-70ee3d78e4ff/data/ERR5327198_01_L001_R1_001.fastq.gz

This is cutadapt 5.1 with Python 3.10.14
Command line parameters: -u 0 -

# 03 Denoising

### 1. Denoising Versuch mit trim left 0 und trunc len 0

In [11]:
!qiime dada2 denoise-single \
   --i-demultiplexed-seqs $data_dir/fungut_forward_reads.qza \
   --p-trim-left 0 \
   --p-trunc-len 0 \
   --p-min-fold-parent-over-abundance 4 \
   --p-max-ee 4 \
    --o-representative-sequences $data_dir/dada2_rep_seqs_1.qza \
    --o-table $data_dir/dada2_table_1.qza \
    --o-denoising-stats $data_dir/dada2_stats_1.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: fungut_data/dada2_table_1.qza
Saved FeatureData[Sequence] to: fungut_data/dada2_rep_seqs_1.qza
Saved SampleData[DADA2Stats] to: fungut_data/dada2_stats_1.qza


In [12]:
! qiime metadata tabulate \
    --m-input-file $data_dir/dada2_stats_1.qza \
    --o-visualization $data_dir/dada2_stats_1.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: fungut_data/dada2_stats_1.qzv


In [13]:
Visualization.load(f"{data_dir}/dada2_stats_1.qzv")

<visualization: Visualization uuid: 9089797b-56d2-4839-80de-ade4e6d423d5>

In [14]:
! qiime feature-table tabulate-seqs \
    --i-data $data_dir/dada2_rep_seqs_1.qza \
    --o-visualization $data_dir/dada2_rep_seqs_1.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: fungut_data/dada2_rep_seqs_1.qzv


In [15]:
Visualization.load(f"{data_dir}/dada2_rep_seqs_1.qzv")

<visualization: Visualization uuid: e8071f18-285a-4b5b-98c5-9a5143a51843>

In [16]:
! qiime feature-table summarize \
    --i-table $data_dir/dada2_table_1.qza \
    --m-sample-metadata-file $data_dir/fungut_metadata.tsv \
    --o-visualization $data_dir/dada2_table_1.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: fungut_data/dada2_table_1.qzv


In [17]:
Visualization.load(f"{data_dir}/dada2_table_1.qzv")

<visualization: Visualization uuid: 019f5bf9-11ae-42bc-be76-620732aa8db8>

### 2. Denoising Versuch mit trim left 15 und trunc len 130

In [ ]:
!qiime dada2 denoise-single \
   --i-demultiplexed-seqs $data_dir/fungut_forward_reads.qza \
   --p-trim-left 15 \
   --p-trunc-len 130 \
   --p-min-fold-parent-over-abundance 4 \
   --p-max-ee 4 \
    --o-representative-sequences $data_dir/dada2_rep_seqs_2.qza \
    --o-table $data_dir/dada2_table_2.qza \
    --o-denoising-stats $data_dir/dada2_stats_2.qza

In [ ]:
! qiime metadata tabulate \
    --m-input-file $data_dir/dada2_stats_2.qza \
    --o-visualization $data_dir/dada2_stats_2.qzv

In [ ]:
Visualization.load(f"{data_dir}/dada2_stats_2.qzv")

In [ ]:
! qiime feature-table tabulate-seqs \
    --i-data $data_dir/dada2_rep_seqs_2.qza \
    --o-visualization $data_dir/dada2_rep_seqs_2.qzv

In [ ]:
Visualization.load(f"{data_dir}/dada2_rep_seqs_2.qzv")

In [ ]:
! qiime feature-table summarize \
    --i-table $data_dir/dada2_table_2.qza \
    --m-sample-metadata-file $data_dir/fungut_metadata.tsv \
    --o-visualization $data_dir/dada2_table_2.qzv

In [ ]:
Visualization.load(f"{data_dir}/dada2_table_2.qzv")

Comparing denoised data 1&2:
ITS sequences are more variable in length than 16S data. There is the risk of cutting the truncation length too short and loose valid ITS reads. Regarding that the the original data was good overall (everywhere over 30), it is better to keep all sequences. With this conclusion, it is decided to continue further steps with the data optained by run 1.

In [ ]:
! qiime feature-classifier classify-sklearn ?


Denoising with trimming

In [ ]:
!qiime dada2 denoise-single \
   --i-demultiplexed-seqs $data_dir/fungut_forward_reads_trimmed.qza \
   --p-trim-left 0 \
   --p-trunc-len 0 \
   --p-min-fold-parent-over-abundance 4 \
   --p-max-ee 4 \
    --o-representative-sequences $data_dir/rep_seqs_trimmed.qza \
    --o-table $data_dir/table_trimmed.qza \
    --o-denoising-stats $data_dir/stats_trimmed.qza

In [ ]:
! qiime feature-table tabulate-seqs \
    --i-data $data_dir/rep_seqs_trimmed.qza \
    --o-visualization $data_dir/rep_seqs_trimmed.qzv

In [ ]:
Visualization.load(f"{data_dir}/rep_seqs_trimmed.qzv")

# 04 Taxonomy

### 04.01 Taxonomy using pretrained classifier

In [ ]:
classifier = unite_ver10_dynamic_s_all_19.02.2025-Q2-2024.10.qza

In [ ]:
! qiime metadata tabulate \
    --m-input-file $data_dir/taxonomy.qza \
    --o-visualization $data_dir/taxonomy.qzv

In [6]:
Visualization.load(f"{data_dir}/taxonomy.qzv")

<visualization: Visualization uuid: 6bf36f99-6d2a-49a1-b3f0-54cc92e709ff>

In [ ]:
euler:
    miniconda und chiime auf euler laden
    jetzt könne wir mit euler kommunizieren
    für jeden job die parameter definieren + wo sollen datein abgespeichert werden
    laufen lassen
    Hat es funktioniert?

In [ ]:
conda 
-> ampicon distribution (nicht m)
transferring files
zeit: 24h
cpu: nach datenbank orientieren -> etwas mehr als das
